In [ ]:
import tensorflow as tf

import os
import tensorflow_datasets as tfds

In [ ]:
from google.colab  import drive
drive.mount("/gdrive")
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
directory_path='/gdrive/My Drive/Yedek/Video_Dataset_hsv'

In [ ]:
subset_paths={'train':directory_path+'/train','test':directory_path+'/test'}

In [ ]:
!pip install hickle
import hickle as hkl
import random
from pathlib import Path
import os
import cv2
import matplotlib.pyplot as plt
from random import randint
import numpy as np
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
def frames_from_video_file(video_path):
  result = []
  src = cv2.VideoCapture(str(video_path))

  for _ in range(1,20,9):
    ret, frame = src.read()
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    tf.image.resize(frame, [224,224])
    result.append(frame)

  src.release()
  result = np.array(result)[..., [2, 1, 0]]
  return result

In [ ]:
class FrameGenerator:
  def __init__(self, path, training = False):

    self.path = path
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.avi'))
    classes = [p.parent.name for p in video_paths]
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path)
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

In [ ]:
train_path = Path(subset_paths['train'])
test_path = Path(subset_paths['test'])

In [ ]:
fg = FrameGenerator(train_path, training=True)
print(fg.class_ids_for_name)

{'healthy': 0, 'sick': 1}


In [ ]:
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.float32))

In [ ]:
train_ds = tf.data.Dataset.from_generator(FrameGenerator(train_path, training=True),
                                          output_signature = output_signature)
test_ds = tf.data.Dataset.from_generator(FrameGenerator(test_path),
                                          output_signature = output_signature)

In [ ]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.front = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(480,640,3)),
            tf.keras.layers.Normalization(),
            tf.keras.layers.Conv2D(16, 3, strides=1, padding='same', activation='relu'),
            tf.keras.layers.MaxPooling2D(2, strides=2),
            tf.keras.layers.Conv2D(8, 3, strides=1, padding='same', activation='relu'),
            tf.keras.layers.MaxPooling2D(2, strides=2),
            tf.keras.layers.Conv2D(4, 3, strides=1, padding='same', activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(4,activation='sigmoid')])

    def call(self, x):
        out = self.front(x)

        return out

In [ ]:
def build_mobilenet(shape=(224, 224, 3), nbout=1):
    model = tf.keras.applications.mobilenet.MobileNet(
        include_top=False,
        input_shape=shape,
        weights='imagenet')

    trainable = 9
    for layer in model.layers[:-trainable]:
        layer.trainable = False
    for layer in model.layers[-trainable:]:
        layer.trainable = True
    output = tf.keras.layers.GlobalMaxPool2D()
    return tf.keras.Sequential([model, output])

In [ ]:
def action_model(convnet):
   model = tf.keras.Sequential()
   model.add(tf.keras.layers.TimeDistributed(convnet, input_shape=(3,224,224,3)))
   model.add(tf.keras.layers.LSTM(4)),
   model.add(tf.keras.layers.Dense(4, activation='relu')),
   model.add(tf.keras.layers.Dense(2, activation='relu')),
   model.add(tf.keras.layers.Dropout(0.2))
   model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
   return model

In [ ]:
convnet=build_mobilenet()
model=action_model(convnet)
model.compile(optimizer = tf.keras.optimizers.SGD(),
              loss='binary_crossentropy', metrics=tf.keras.metrics.BinaryAccuracy(name='binary_accuracy', dtype=None, threshold=0.5))

17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
train_ds = train_ds.batch(8)
test_ds = test_ds.batch(8)

train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

test_frames, test_labels = next(iter(test_ds))
print(f'Shape of validation set of frames: {test_frames.shape}')
print(f'Shape of validation labels: {test_labels.shape}')

Shape of training set of frames: (8, 3, 480, 640, 3)
Shape of training labels: (8,)
Shape of validation set of frames: (8, 3, 480, 640, 3)
Shape of validation labels: (8,)


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)

In [ ]:
epochs=20

In [ ]:
history=model.fit( train_ds, epochs=epochs)

Epoch 1/20
22/22 [==============================] - 154s 308ms/step - loss: 0.7250 - binary_accuracy: 0.4375
Epoch 2/20
22/22 [==============================] - 7s 306ms/step - loss: 0.7095 - binary_accuracy: 0.4091
Epoch 3/20
22/22 [==============================] - 7s 306ms/step - loss: 0.6946 - binary_accuracy: 0.4375
Epoch 4/20
22/22 [==============================] - 7s 312ms/step - loss: 0.6864 - binary_accuracy: 0.6364
Epoch 5/20
22/22 [==============================] - 7s 313ms/step - loss: 0.6778 - binary_accuracy: 0.6364
Epoch 6/20
22/22 [==============================] - 7s 317ms/step - loss: 0.6742 - binary_accuracy: 0.6364
Epoch 7/20
22/22 [==============================] - 7s 310ms/step - loss: 0.6723 - binary_accuracy: 0.6364
Epoch 8/20
22/22 [==============================] - 7s 309ms/step - loss: 0.6706 - binary_accuracy: 0.6364
Epoch 9/20
22/22 [==============================] - 7s 306ms/step - loss: 0.6690 - binary_accuracy: 0.6364
Epoch 10/20
22/22 [================

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_ds ,return_dict=True)